In [1]:
import pandas as pd
import numpy as np
from mapping import Mapping
import time
fig_prefix = "../../figuires/2018-07-22-jw-weaselball_analysis"
data_prefix = "../../data/2018-07-22-jw-weaselball_analysis_"

In [2]:
start_time = time.time()
FLOAT_ERROR_TOLERANCE = 0.00000000001 #See IEEE 754 for why a floating point is never perfect
df_strings = ['../../data/08-11-2018_17-59-32.csv', '../../data/08-11-2018_17-59-39.csv']
frames = []
for csv in df_strings:
    temp = pd.read_csv(csv,index_col=False )
    frames.append(temp)
df = pd.concat(frames,ignore_index=True)
print(df.shape)
df = df.drop(columns=['Time'])
df = df.dropna() #Get rid of any rows with NA in it.
df = df.apply(pd.to_numeric)
df.head(10)

(19302242, 8)


,ID,X,Y,Yaw,ResetID,checkCorrectness,NumberOfWalls
0,0,0.000000,0.000000,0.00000,1,0,0
1,0,-0.466000,-0.279600,-2.29649,2,1,0
2,0,-0.466001,-0.279601,-2.29649,2,1,0
3,0,-0.466001,-0.279601,-2.29649,2,1,0
4,0,-0.466001,-0.279601,-2.29649,2,1,0
5,0,-0.466001,-0.279601,-2.29649,2,1,0
6,0,-0.466001,-0.279601,-2.29649,2,1,0
7,0,-0.466001,-0.279601,-2.29649,2,1,0
8,0,-0.466001,-0.279601,-2.29649,2,1,0
9,0,-0.466001,-0.279601,-2.29649,2,1,0


In [3]:
df.tail(10)

,ID,X,Y,Yaw,ResetID,checkCorrectness,NumberOfWalls
19302232,0,0.311417,0.486632,1.19492,18,1,0
19302233,0,0.311421,0.486478,1.19486,18,1,0
19302234,0,0.311424,0.486322,1.19479,18,1,0
19302235,0,0.311427,0.486168,1.19472,18,1,0
19302236,0,0.311431,0.486016,1.19465,18,1,0
19302237,0,0.311432,0.485866,1.19458,18,1,0
19302238,0,0.311431,0.485714,1.19451,18,1,0
19302239,0,0.311430,0.485563,1.19445,18,1,0
19302240,0,0.311434,0.485405,1.19438,18,1,0
19302241,0,0.311436,0.485250,1.19432,18,1,0


In [4]:
#Sample the data
SAMPLING_RATE = 10 #Keep 1 row for every SAMPLING_RATE
df_sampled = df.iloc[::SAMPLING_RATE,:]
print("Size of new DF is {}".format(df_sampled.shape))
df_sampled.head(10)

Size of new DF is (1930225, 7)


,ID,X,Y,Yaw,ResetID,checkCorrectness,NumberOfWalls
0,0,0.000000,0.000000,0.00000,1,0,0
10,0,-0.466001,-0.279601,-2.29649,2,1,0
20,0,-0.466003,-0.279603,-2.29649,2,1,0
30,0,-0.466228,-0.279134,-2.29628,2,1,0
40,0,-0.466557,-0.278341,-2.29608,2,1,0
50,0,-0.466791,-0.277501,-2.29580,2,1,0
60,0,-0.466929,-0.276661,-2.29544,2,1,0
70,0,-0.466965,-0.275844,-2.29504,2,1,0
80,0,-0.466910,-0.274979,-2.29471,2,1,0
90,0,-0.466941,-0.274237,-2.29436,2,1,0


In [5]:
#Break data into 3 parts. S = {Near 2 Walls, Near 1 Wall, Near No Walls}
#df2 = df_sampled.loc[df_sampled['ResetID'] % 3 == 2]
#df1 = df_sampled.loc[df_sampled['ResetID'] % 3 == 1]
#df0 = df_sampled.loc[df_sampled['ResetID'] % 3 == 0]
#df = df0

In [6]:
#Clean up the data
df_clean = df_sampled.copy()

df_clean.index = range(df_clean.shape[0])
#When the gazebo run it may collect some data of the robots when they aren't moving for the first few 50 or so samples.
#TODO
df_clean.head()

,ID,X,Y,Yaw,ResetID,checkCorrectness,NumberOfWalls
0,0,0.000000,0.000000,0.00000,1,0,0
1,0,-0.466001,-0.279601,-2.29649,2,1,0
2,0,-0.466003,-0.279603,-2.29649,2,1,0
3,0,-0.466228,-0.279134,-2.29628,2,1,0
4,0,-0.466557,-0.278341,-2.29608,2,1,0


In [7]:
#Clean up the data

#Shift Yaw to go from 0 to 2pi, so just add PI since it currently goes for -pi to pi

df_clean['Yaw'] += np.pi
#The following is added so we can better discretize into 0, 180, 360, 480 degrees
#df_clean['Yaw'] -= (np.pi / 4.0)
#if(df_clean['Yaw'].max() > 2 * np.pi or df_clean['Yaw'].min() < 0):
#    print("[ERROR] Cleaning Yaw failed. Make sure range is from 0 and 2 * pi")
#    print("Yaw = ({} - {})".format(df_clean['Yaw'].min(), df_clean['Yaw'].max()))


df_clean.describe()

,ID,X,Y,Yaw,ResetID,checkCorrectness,NumberOfWalls
count,1930225.0,1.930225e+06,1.930225e+06,1.930225e+06,1.930225e+06,1.930225e+06,1.930225e+06
mean,0.0,-1.878898e-03,6.248936e-04,3.245924e+00,9.530935e+00,9.999995e-01,3.116372e-02
std,0.0,3.284680e-01,3.310730e-01,1.797157e+00,4.629760e+00,7.197738e-04,1.745246e-01
min,0.0,-5.016580e-01,-5.014010e-01,2.653590e-06,1.000000e+00,0.000000e+00,0.000000e+00
25%,0.0,-3.129390e-01,-3.114270e-01,1.810883e+00,6.000000e+00,1.000000e+00,0.000000e+00
50%,0.0,-9.679170e-04,-2.911000e-04,3.325317e+00,1.000000e+01,1.000000e+00,0.000000e+00
75%,0.0,3.048460e-01,3.147540e-01,4.815593e+00,1.400000e+01,1.000000e+00,0.000000e+00
max,0.0,5.015260e-01,5.010800e-01,6.283183e+00,1.800000e+01,1.000000e+00,2.000000e+00


In [8]:
#Clean up the data

#Shift X and Y over so that way it could be made easier to use. Currently the world reference is at the center of the "play area"
LENGTH_OF_BOX = 1.127 #This can be obtained from the .sdf file of the weazelball enclosure in gazebo


df_clean['X'] += LENGTH_OF_BOX / 2
df_clean['Y'] += LENGTH_OF_BOX / 2
if (df_clean['X'].max() > LENGTH_OF_BOX or df_clean['Y'].max() > LENGTH_OF_BOX or df_clean['X'].min() < 0 or df_clean['Y'].min() < 0):
    print("[ERROR] Cleaning X/Y failed, Make sure the points are between 0 and LENGTH_OF_BOX")
    print("Y = ({} - {}) X = ({} to {})".format(df_clean['Y'].min(), df_clean['Y'].max(), df_clean['X'].min(), df_clean['X'].max()))
df_clean.head()


,ID,X,Y,Yaw,ResetID,checkCorrectness,NumberOfWalls
0,0,0.563500,0.563500,3.141593,1,0,0
1,0,0.097499,0.283899,0.845103,2,1,0
2,0,0.097497,0.283897,0.845103,2,1,0
3,0,0.097272,0.284366,0.845313,2,1,0
4,0,0.096943,0.285159,0.845513,2,1,0


In [9]:
#Clean up the data

#Clean the time data since gazebo prints it weird...
#TODO

In [10]:
#Discretize the data
NUMBER_OF_SQUARES = 1 #This should be a square number to create equal sized squares.
RESOLUTION_OF_S1 = 0.1 #This is used to discretize the yaw angle over 0 - 2*pi

df_discretized = df_clean.copy()
    
mappingBoxConstant = (NUMBER_OF_SQUARES ** (1/2.0)) / (LENGTH_OF_BOX)
for index, row in df_clean.iterrows():
    df_discretized.at[index, 'X'] = int(row['X'] * mappingBoxConstant)
    df_discretized.at[index, 'Y'] = int(row['Y'] * mappingBoxConstant)
    df_discretized.at[index, 'Yaw'] = int(row['Yaw'] / RESOLUTION_OF_S1) * RESOLUTION_OF_S1
        
df_discretized.describe()

,ID,X,Y,Yaw,ResetID,checkCorrectness,NumberOfWalls
count,1930225.0,1930225.0,1930225.0,1.930225e+06,1.930225e+06,1.930225e+06,1.930225e+06
mean,0.0,0.0,0.0,3.195934e+00,9.530935e+00,9.999995e-01,3.116372e-02
std,0.0,0.0,0.0,1.797113e+00,4.629760e+00,7.197738e-04,1.745246e-01
min,0.0,0.0,0.0,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00
25%,0.0,0.0,0.0,1.800000e+00,6.000000e+00,1.000000e+00,0.000000e+00
50%,0.0,0.0,0.0,3.300000e+00,1.000000e+01,1.000000e+00,0.000000e+00
75%,0.0,0.0,0.0,4.800000e+00,1.400000e+01,1.000000e+00,0.000000e+00
max,0.0,0.0,0.0,6.200000e+00,1.800000e+01,1.000000e+00,2.000000e+00


In [11]:
#Verify Discretizing suceeded by checking that number of states generated is the number of states we expeted or less (Sometimes these things dont visit all states)

if (df_discretized['X'].max() > (NUMBER_OF_SQUARES ** (1/2.0)) or df_discretized['Y'].max() > (NUMBER_OF_SQUARES ** (1/2.0)) or df_discretized['X'].min() < 0 or df_discretized['Y'].min() < 0):
    print("[ERROR] Discretizing X/Y failed, Make sure the points are between 0 and (NUMBER_OF_SQUARES ** (1/2.0)")
    print("Y = ({} - {}) X = ({} to {})".format(df_clean['Y'].min(), df_clean['Y'].max(), df_clean['X'].min(), df_clean['X'].max()))


In [12]:
TRANSLATION_MATRIX_INITIAL_VALUE = 1
#Create the matrix representing the Markov Chain
#I am assuming we are discretizing the space into equal sized boxes
#The transition matrix A is of size 
#( # of states of X * # of states of Y * # of states of Yaw)
#( # of states of X and Y = mappingBoxConstant * LENGTH_OF_BOX )
#( # of states of Yaw = int(2*pi / RESOLUTION_OF_S1)+1)
number_of_x_states = mappingBoxConstant * LENGTH_OF_BOX
number_of_y_states = mappingBoxConstant * LENGTH_OF_BOX
number_of_s1_states = int(2*np.pi / RESOLUTION_OF_S1) + 1

n = int(number_of_x_states * number_of_y_states * number_of_s1_states)
print("[DEBUG] Size of n is {}".format(n))
translation_matrix = pd.DataFrame(0, index=range(n), columns=range(n))#We use 1 here to set the whole matrix elements to 1
translation_matrix.head()

[DEBUG] Size of n is 63


,0,1,2,3,4,5,6,7,8,9,...,53,54,55,56,57,58,59,60,61,62
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
print(number_of_s1_states)

63


In [14]:


#The formula for mapping a 3D array to 1D is
#(z * xMax * yMax) + (y * xMax) + x;
#https://stackoverflow.com/questions/7367770/how-to-flatten-or-index-3d-array-in-1d-array
X_MAX = NUMBER_OF_SQUARES ** (1/2.0)
Y_MAX = NUMBER_OF_SQUARES ** (1/2.0)
YAW_MAX = (2 * np.pi) / RESOLUTION_OF_S1
YAW_MAX += 0
mapping = Mapping(X_MAX, Y_MAX, YAW_MAX)#Fill in the logical areas that the system can reach (For now I am assuming it can go up/down 2 yaw states or the surronding (x,y) blocks)
print("XMAX = {}, YMAX = {} YAWMAX = {}".format(X_MAX,Y_MAX,YAW_MAX))

XMAX = 1.0, YMAX = 1.0 YAWMAX = 62.83185307179586


In [15]:
#HUERISTIC: Add a +1 to any logical possible state the structure would likely end up in.
#This lowers the amount of artifiical data in the matrix (most of which isnt needed)



for index, row in translation_matrix.iterrows():
    if(TRANSLATION_MATRIX_INITIAL_VALUE == 0):
        break
    (x,y,yaw) = mapping.map1Dto3D(index)
    #Generate all possible (x,y,yaw) permutations
    #I assume we can only move with 1 around (x,y) including diagnols and +- 2 yaw states
    VARIATION_OF_X = 1
    VARIATION_OF_Y = 1
    VARIATION_OF_YAW = 1
    possible_spots = []
    for x_p in range(-VARIATION_OF_X,VARIATION_OF_X+1):
        for y_p in range(-VARIATION_OF_Y,VARIATION_OF_Y+1):
            for yaw_p in range(-VARIATION_OF_YAW,VARIATION_OF_YAW+1):
                if(mapping.checkValid3DMap(x+x_p, y+y_p, yaw+yaw_p)):
                    possible_spots.append( (x+x_p, y+y_p, yaw+yaw_p) )
    for pose in possible_spots:
        translation_matrix.at[mapping.map3Dto1D(x,y,yaw), mapping.map3Dto1D(pose[0],pose[1],pose[2])] = translation_matrix.at[mapping.map3Dto1D(x,y,yaw), mapping.map3Dto1D(pose[0],pose[1],pose[2])] +TRANSLATION_MATRIX_INITIAL_VALUE
translation_matrix.head()    

,0,1,2,3,4,5,6,7,8,9,...,53,54,55,56,57,58,59,60,61,62
0,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,1,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,1,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,1,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [16]:
#Create a dictionary for storing the transition states analysis
d ={}
#Our keys to the dictionary will look like (x_t, y_t, yaw_t, x_t+1, y_t+1, yaw_t+1)
#Go through all but last row since t+1 isnt defined there...
skipCount = 0
try:
    for index, row in df_discretized.iterrows():
        if(index == df_discretized.index[-1]):
            break
        if(df_discretized.at[index, 'ResetID'] != df_discretized.at[index + 1, 'ResetID']):
            skipCount += 1
            continue
        #Need to round here because Yaw data has floating point error
        key = (df_discretized.at[index, 'X'], df_discretized.at[index, 'Y'], round(df_discretized.at[index, 'Yaw'], 6),df_discretized.at[index+1, 'X'], df_discretized.at[index+1, 'Y'], round(df_discretized.at[index+1, 'Yaw'], 6) )

        if key in d:
            d[key] += 1
        else:
            d[key] = 1
except Exception as e:
    print (e)
    
print("[DEBUG] Skipped {} events".format(skipCount))
d

[DEBUG] Skipped 34 events


{(0.0, 0.0, 0.8, 0.0, 0.0, 0.8): 22326,
 (0.0, 0.0, 0.8, 0.0, 0.0, 0.7): 275,
 (0.0, 0.0, 0.7, 0.0, 0.0, 0.7): 23516,
 (0.0, 0.0, 0.7, 0.0, 0.0, 0.6): 280,
 (0.0, 0.0, 0.6, 0.0, 0.0, 0.6): 24326,
 (0.0, 0.0, 0.6, 0.0, 0.0, 0.7): 401,
 (0.0, 0.0, 0.6, 0.0, 0.0, 0.5): 308,
 (0.0, 0.0, 0.5, 0.0, 0.0, 0.5): 31903,
 (0.0, 0.0, 0.5, 0.0, 0.0, 0.4): 434,
 (0.0, 0.0, 0.4, 0.0, 0.0, 0.4): 44572,
 (0.0, 0.0, 0.4, 0.0, 0.0, 0.3): 434,
 (0.0, 0.0, 0.3, 0.0, 0.0, 0.3): 31666,
 (0.0, 0.0, 0.3, 0.0, 0.0, 0.2): 320,
 (0.0, 0.0, 0.2, 0.0, 0.0, 0.2): 23070,
 (0.0, 0.0, 0.2, 0.0, 0.0, 0.1): 287,
 (0.0, 0.0, 0.1, 0.0, 0.0, 0.1): 20776,
 (0.0, 0.0, 0.1, 0.0, 0.0, 0.0): 277,
 (0.0, 0.0, 0.0, 0.0, 0.0, 0.0): 23124,
 (0.0, 0.0, 0.0, 0.0, 0.0, 6.2): 282,
 (0.0, 0.0, 6.2, 0.0, 0.0, 6.2): 22192,
 (0.0, 0.0, 6.2, 0.0, 0.0, 6.1): 310,
 (0.0, 0.0, 6.1, 0.0, 0.0, 6.1): 26658,
 (0.0, 0.0, 6.1, 0.0, 0.0, 6.0): 354,
 (0.0, 0.0, 6.0, 0.0, 0.0, 6.0): 31567,
 (0.0, 0.0, 6.0, 0.0, 0.0, 5.9): 443,
 (0.0, 0.0, 5.9, 0.0, 0.0,

In [17]:

#Fill in matrix with dictionary data


for key, value in d.items():
    #we need to map yaw to an int state
    element_t = mapping.map3Dto1D(key[0], key[1], int(key[2]/RESOLUTION_OF_S1))
    element_t_plus_1 = mapping.map3Dto1D(key[3], key[4], int(key[5]/RESOLUTION_OF_S1))
    #Use the following to verify we the math above is fine
    if((mapping.checkValid1DMap(element_t)) & (mapping.checkValid1DMap(element_t_plus_1)) == 0 ):
        print("[ERROR] BAD MAPPING!")
    try:
        translation_matrix.at[element_t, element_t_plus_1] = value + translation_matrix.at[element_t, element_t_plus_1]
    except Exception as e:
        print("error = {}".format(e))
        print("element _t = {}".format(element_t))
        print("element _t = {}".format(mapping.map1Dto3D(element_t)))
        print("element_t+1 = {}".format(element_t_plus_1))
        print("element _t+1 = {}".format(mapping.map1Dto3D(element_t_plus_1)))
    # print("key = {}, elements = {}, {}".format(key, element_t, element_t_plus_1))


[ERROR] BAD MAPPING!
[ERROR] BAD MAPPING!
[ERROR] BAD MAPPING!
[ERROR] BAD MAPPING!
[ERROR] BAD MAPPING!


In [18]:
foo = d.keys()
from operator import itemgetter
min(foo,key=itemgetter(2))

(0.0, 0.0, 0.0, 0.0, 0.0, 0.0)

In [19]:
#Check sum of "events" per matrix

totalEvents = 0
for index,row in translation_matrix.iterrows():
    totalEvents += row.sum()
print("Total Events is {}".format(totalEvents))
print("Size of data point df is {}".format(df_discretized.size))

Total Events is 1930375
Size of data point df is 13511575


In [20]:
#Divide the whole dataframe by number of data collections to get the probabilities.


magnitudeVector = pd.Series(0, index=range(n + 1))



for index, row in translation_matrix.iterrows():
    totalActionsInThisState = row.sum()
    magnitudeVector.iloc[index] = totalActionsInThisState
    if totalActionsInThisState == 0:
        continue
    translation_matrix.iloc[index] /= totalActionsInThisState

translation_matrix.head()

,0,1,2,3,4,5,6,7,8,9,...,53,54,55,56,57,58,59,60,61,62
0,0.971353,0.016802,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.011845
1,0.012952,0.967993,0.019055,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
2,0.000000,0.005112,0.985003,0.000018,0.009868,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
3,0.000000,0.000000,0.333333,0.333333,0.333333,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
4,0.000000,0.000000,0.009525,0.000022,0.978229,0.012224,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000


In [21]:
#validate the matrix (all rows == 1)
for index, row in translation_matrix.iterrows():
    if(abs(row.sum() - 1.0) > FLOAT_ERROR_TOLERANCE):
        print("[ERROR] Row probability not equal to one!")
        print(index)
        print(row.sum())


In [22]:
#Make matrix into CSV
translation_matrix.to_csv(data_prefix + 'markov4_100.csv', encoding='utf-8', index=False)


In [23]:
#Make csv of the number of instances for each row
magnitudeVector.to_csv(data_prefix + 'magnitude_vector_out.csv', encoding='utf-8', index=False)


In [24]:
print("Total time taken = {}".format(time.time()-start_time))

Total time taken = 621.1889054775238
